## Decorators

A common pattern in functional programs, are functions that are built to "wrap" other functions.

Functions are in fact mutable.

In [5]:
def print_before_and_after(func):
    def newfunc(*args, **kwargs):
        print("BEFORE", func)
        func(*args, **kwargs)
        print("AFTER", func)
    return newfunc

In [8]:
def inner(a, b, c):
    print("inner function", a, b, c)

In [10]:
wrapped_inner = print_before_and_after(inner)

wrapped_inner(1, 2, 3)
#print(x)

BEFORE <function inner at 0x103591240>
inner function 1 2 3
AFTER <function inner at 0x103591240>


In [11]:
# often we want to just replace the function altogether
# with the modified version
inner = print_before_and_after(inner)
inner(1, 2, 3)

BEFORE <function inner at 0x103591240>
inner function 1 2 3
AFTER <function inner at 0x103591240>


In [14]:
# another way of writing this is to use decorator syntax
@print_before_and_after
@print_before_and_after
def add_nums(a, b, c):
    print(f"{a} + {b} + {c} =", a + b + c)
# add_nums = print_before_and_after(add_nums)

In [15]:
add_nums(1, 2, 3)

BEFORE <function print_before_and_after.<locals>.newfunc at 0x103591ea0>
BEFORE <function add_nums at 0x1035923b0>
1 + 2 + 3 = 6
AFTER <function add_nums at 0x1035923b0>
AFTER <function print_before_and_after.<locals>.newfunc at 0x103591ea0>


In [1]:
def cache(func):
    inner_cache = {}
    
    def newfunc(*args, **kwargs):
        if args not in inner_cache:
            # will not work correctly if there are kwargs
            inner_cache[args] = func(*args, **kwargs)
        return inner_cache[args]
    
    return newfunc

In [ ]:
@cache
def expensive_calculation(a, b, *, c=0):
    print(f"doing expensive calculation on {a} {b}...")
    return a ** b
#expensive_calculation = cache(expensive_calculation)

@cache
def cheap_calculation(a, b):
    print(f"doing cheap calculation on {a} {b}...")
    return a + b

In [ ]:
expensive_calculation(4, 10)

In [ ]:
expensive_calculation(4, 10)

In [ ]:
cheap_calculation(4, 10)

In [ ]:
expensive_calculation(5, 6)

In [ ]:
cheap_calculation(4, 10)

In [ ]:
expensive_calculation(5, 6)

In [31]:
def repeat5(func):                     # the decorator   
    def newfunc(a, b, c):      # the inner function
        return func(a, b, c, c, b, a)
    return newfunc

@repeatN
def print_sum(*args):
    print(sum(args))


TypeError: repeatN() missing 1 required positional argument: 'n'

In [29]:
print_sum(1, 2, 3)

6
6
6
6
6


In [10]:
# to make a decorator that takes additional arguments
# you need to write a decorator factory function that returns decorators

total = 123

def repeat(start, end):  # factory: takes integer, returns decorator
    def repeat_decorator(func):                  # decorator: takes function, returns function
        def newfunc(*args, **kwargs):            # inner function: takes ?, returns ?
            for i in range(total):
                func(*args, **kwargs)
        return newfunc
    return repeat_decorator

@repeat(10)
def print_backwards(s):
    print(s[::-1])

print_backwards("backwards")

sdrawkcab
sdrawkcab
sdrawkcab
sdrawkcab
sdrawkcab
sdrawkcab
sdrawkcab
sdrawkcab
sdrawkcab
sdrawkcab


In [ ]:
repeat_10 = repeat(10)
print(repeat_10)
print_backwards = repeat_10(print_backwards)


In [ ]:
# Example: writing our own `partial`
# https://docs.python.org/3/library/functools.html#functools.partial

In [32]:
import functools
print_hello_names = functools.partial(print, "Hello", sep=", ")

In [33]:
print_hello_names("Scott", "Paul", "Lauren")
# same as print("Hello", "Scott", "Paul", "Lauren", sep=", ")

Hello, Scott, Paul, Lauren


In [17]:
print_hello_names.args

('Hello',)

In [18]:
print_hello_names.keywords

{'sep': ', '}

In [19]:
print_hello_names.func

<function print>

In [36]:
# since functions are objects, we can attach arbitrary values to them
def wrapper(func):
    def newfunc(*args, **kwargs):
        return func(*args, **kwargs)
    # we can do whatever we like after defining newfunc, but before returning it
    newfunc.xyz = "hello"*2
    return newfunc

In [35]:
# property is assigned to all wrapped functions
@wrapper
def our_function():
    print("inside our function")

our_function.xyz

True

In [40]:
def our_partial(func, /, *args, **keywords):
    def newfunc(*fargs, **fkeywords):
        newkeywords = {**keywords, **fkeywords}
        return func(*args, *fargs, **newkeywords)
    # assign these properties from within the closure
    newfunc.func = func
    newfunc.args = args
    newfunc.keywords = keywords
    return newfunc

In [39]:
print_hello_names2 = our_partial(print, "Hello", sep=", ")
print_hello_names2("Scott", "Paul", "Lauren", end="!")

Scott, Paul, Lauren, Hello!

In [37]:
#print_hello_names2 = our_partial(print, "Hello", sep=", ")
print_hello_names2("Scott", "Paul", "Lauren", end="!", sep="?")

Hello?Scott?Paul?Lauren!

In [25]:
print_hello_names2.args

('Hello',)

In [26]:
print_hello_names2.keywords

{'sep': ', '}

In [27]:
print_hello_names2.func

<function print>